In [3]:
from pyspark import SparkContext

In [4]:
sc = SparkContext() 

In [5]:
sc

# 11장 Machine Learning with MLlib

 - MLib의 디장인과 철학은 단순 : RDD를 사용하는 ML함수 집합
 
 - ML을 이용한 text classification 작업의 예
     - 1) 메시지를 표현하는 문자열들이 담긴 RDD를 준비 
     - 2) MLlib 특성 추출 알고리즘들 중 하나를 써서 문자열을 수치화된 특성으로 변환을 통한 결과 벡터들의 RDD리턴
     - 3) 벡터의 RDD에 분류 알고리즘을 호출, 신규 데이터들을 분류하는데 쓰일 모델 객체 리턴
     - 4) MLlib평가 함수들 중 하나를 써서 테스트 데이터 세트에 모델을 적용, 평가

### 시스템 요구 사항
- MLlib은 시스템에 몇몇 선형대수 라이브러리 설치를 필요로 함. 만약 해당 오류 발생시 우선 OS에 gfortran 실행 라이브러리 설치
- Dependencies
   - 스칼라, 자바 : breeze 
   - 파이션 : numpy
- 우리의 환경 설정 : docker pull jupyter/all-spark-notebook

### 머신러닝의 기초
- 머신러닝 알고리즘은 훈련 데이터를 통한 알고리즘의 수학적 결과들을 토대로 예측, 분류등에 대한 결정을 내리려고 시도함.
- 머신러닝의 일반적인 단계 : 
    - 문제정의 -> 변수추출 -> 정제 및 변환 -> 분석 -> 측정
    
![](spark11_01.png)

### 스팸분류 예제 

In [10]:
! wget https://raw.githubusercontent.com/biospin/BigBio/master/part03/week04_160525/spark/spam.txt

--2016-05-23 01:33:04--  https://raw.githubusercontent.com/biospin/BigBio/master/part03/week04_160525/spark/spam.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 103.245.222.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|103.245.222.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16144 (16K) [text/plain]
Saving to: ‘spam.txt’

spam.txt            100%[=====================>]  15.77K  --.-KB/s   in 0.03s  

2016-05-23 01:33:14 (458 KB/s) - ‘spam.txt’ saved [16144/16144]



In [11]:
! wget https://raw.githubusercontent.com/biospin/BigBio/master/part03/week04_160525/spark/normal.txt

--2016-05-23 01:33:47--  https://raw.githubusercontent.com/biospin/BigBio/master/part03/week04_160525/spark/normal.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 103.245.222.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|103.245.222.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17295 (17K) [text/plain]
Saving to: ‘normal.txt’

normal.txt          100%[=====================>]  16.89K  --.-KB/s   in 0.06s  

2016-05-23 01:33:53 (276 KB/s) - ‘normal.txt’ saved [17295/17295]



In [13]:
! ls -l

total 44
-rw-r--r-- 1 jovyan users  4215 May 23 01:34 learning.spark.11.MachineLearningwithMLlib.ipynb
-rw-r--r-- 1 jovyan users 17295 May 23 01:33 normal.txt
-rw-r--r-- 1 jovyan users 16144 May 23 01:33 spam.txt


In [14]:
from pyspark.mllib.regression import LabeledPoint 
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD

In [15]:
spam = sc.textFile( "spam.txt" )
normal = sc.textFile( "normal.txt" )

In [16]:
# 이메일 문자열을 10000개의 특징의 벡터로 매핑하기 위한 HashingTF 인스턴스를 만듬.
tf = HashingTF( numFeatures=10000)

In [18]:
# 각 이메일을 단어로 구분하고, 각 단어는 하나의 특성에 패밍
spamFeatures = spam.map( lambda email : tf.transform( email.split(" ") )  )
normalFeatures = normal.map( lambda email : tf.transform( email.split(" ") )  )

In [23]:
# 양성( spam )과 음성( 일반 ) 예제를 위한 LabeledPoint( 1, features ) 데이터세을 만듬.
positiveExamples = spamFeatures.map( lambda features : LabeledPoint( 1, features ) ) 
negativeExamples = normalFeatures.map( lambda features : LabeledPoint( 0, features ) ) 
trainingData = positiveExamples.union( negativeExamples   )
trainingData.cache()

UnionRDD[15] at union at NativeMethodAccessorImpl.java:-2

In [24]:
# SGD 알고리즘으로 로직스틱 회귀를 수행
model = LogisticRegressionWithSGD.train(trainingData)

In [29]:
# 양성예제와 음성예제를 테스트한다.
# 우선, 벡터들을 얻기 위해 동일한 HashingTF 특징 변형을 적용한 후 모델을 적용함.
posTest = tf.transform( "Why not grab out amazing Python Data Bundle for only $50. Pick up our Python Data Bundle for $50 Purchase Bundle View all eBooks & Videos".split(" ") )
negTest = tf.transform( "Research Center developed a highly optimized matrix factorization system to accelerate recommendations for e-commerce sites".split(" ") )


print "Prediction for positive test example:", model.predict(posTest)
print "Prediction for negative test example:", model.predict(negTest)

Prediction for positive test example: 1
Prediction for negative test example: 0


## 데이터 타입

- MLlib에서의 특정 타임들의 패키지 위치
    - 자바, 스칼라 : org.apache.spark.mllib
    - 파이션 : pyspark.mllib
    
 - 주요 데이터 타입
    - Local Vector : 수학적인 의미의 벡터, dense vector와 sparse vector 모두 지원
    - Labeled point : 분류나 회귀같은 지도학습 알고리즘을 위한 Labeled Point
    - Rating : 사용자 상품 순위, 상품 추천을 위한 mllib.recommendation 패키지에서 제공
    - 기타 여러가지 타입이 있음.

In [30]:
from numpy import array
from pyspark.mllib.linalg import Vectors

In [33]:
# 고밀도 백터< 1.0, 2.0, 3.0 > 을 만듬
denseVec1 = array( [1.0, 2.0, 3.0] ) # numpy 배열은 MLlib에 직접 전달
denseVec2 = Vectors.dense( [1.0, 2.0, 3.0] ) # Vector 클래스

In [34]:
# 정밀도 벡터< 1.0, 0.0, 2.0, 0.0 > 을 만듬. 벡터의 크기는 4
sparseVec1 = Vectors.sparse(4, { 0:1.0, 2:2.0 } )
sparseVec2 = Vectors.sparse(4, [0, 2],  [1.0, 2.0] )

## 알고리즘

### 특성추출
    - TF-IDF : 문서에 대한 특성을 추출할때 사용하고, 단어 빈도수와 역문서 빈도를 곱한 값
    - IDF : 전체문서의 수(분자)를 해당 단어가 포함된 문서의 수(분모)로 나눈 값에 로그를 취한 값
    - TF-IDF의 의미는 자주 나오는 단어도 중요하지만, 전체문서에서 드물게 나온 단어가 더 중요함.

In [35]:
! mkdir data 

In [37]:
! cat "Why not grab out amazing  " > data/aaa.txt
! cat "Research Center developed a highly optimize" > data/bbb.txt
! cat "Pick up our Python Data Bundle for" > data/ccc.txt

cat: Why not grab out amazing  : No such file or directory
cat: Research Center developed a highly optimize: No such file or directory
cat: Pick up our Python Data Bundle for: No such file or directory


In [64]:
! ls -l  /home/jovyan/work/data/

total 0
-rw-r--r-- 1 jovyan users 0 May 23 02:37 aaa.txt
-rw-r--r-- 1 jovyan users 0 May 23 02:37 bbb.txt
-rw-r--r-- 1 jovyan users 0 May 23 02:37 ccc.txt


In [41]:
from pyspark.mllib.feature import HashingTF

In [54]:
sentense = "Hello Hello world" 
words = sentense.split()
tf = HashingTF( 10000 )
tf.transform( words )

SparseVector(10000, {3065: 1.0, 7741: 2.0})

In [69]:
rdd = sc.wholeTextFiles("/home/jovyan/work/data").map( lambda(name ,text) : text.split()  )
tfVectors = tf.transform( rdd )
for x in tfVectors.collect():
    print x

(10000,[],[])
(10000,[],[])
(10000,[],[])


### 정량화

- 각각의 특성치들이 스케일이 다르면, 큰 값을 갖는 특성치가 더 많이 적용하게 됨.
- 모든 특성들에 동일하게 고려하기 위해서  정량화가 필요함.
- mllib.feature의 StandardScaler( 평균 : 0,  표준편차 1로 정량화 )

In [72]:
from pyspark.mllib.feature import StandardScaler

In [76]:
vectors = [ Vectors.dense([-2, 5.0, 1.0]), Vectors.dense([2.0, 0.0, 1.0])  ]
dataset = sc.parallelize(vectors)
scaler = StandardScaler( withMean=True, withStd=True)
model = scaler.fit( dataset )
result = model.transform( dataset )
for x in result.collect():
    print x

[-0.707106781187,0.707106781187,0.0]
[0.707106781187,-0.707106781187,0.0]


### 정규화 

- 어떤 상황에서는 벡터를 길이 1로 정규화하는 것이 입력데이터를 준비하는데 도움.
- Normailizer클래스가 이를 담당하여 Normailizer().transform(rdd)를 호출하여 사용

### Word2Vec
- 신경망 네트워크 기반 문자열의 특징을 판별하는 알고리즘.

### 통계 

- mllib.stat.Statisics 클래스의 메소드
- Statisics.coStats( rdd ) 
    - 벡터의 RDD의 통계적인 요약을 계산하며, 최소값, 최대값, 평균, 분산등
- Statisics.corr( rdd, method ) 
    - 벡터의 RDD의 컬럼들ㅇ 사이에 상관관계 행렬을 계산, 피어슨이나 스피어만 상관관계 중 하나를 사용
- Statisics.chiSqTest( rdd ) 
    - LabeledPoint 형태로 추출된 Feature에 대한 피어슨 독립성 검증 제공
    - 결과로 유의확율값, t-통계값 , 각 변수의 자유도를 배열 형태로 제공
기타 : mean, stdev, sum 함수 등 제공